In [1]:
%pwd

'd:\\Computer Engineering\\8 th semester\\final year project\\RAG\\MedicalChatbot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Computer Engineering\\8 th semester\\final year project\\RAG\\MedicalChatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\Amantha\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download Huggingface embedding model

In [4]:
# embedding model
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the hugging face embedding model
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name = model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\Amantha\AppData\Local\Temp\ipykernel_22708\1951281138.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Amantha\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup pinecone

In [5]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

os.environ["PINECOEN_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [6]:
# import pinecone
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

# authenticate pinecone account
pc = Pinecone(api_key=pinecone_api_key)

In [7]:
pc

In [8]:
import os
import time
from pinecone import Pinecone, ServerlessSpec


index_name = "railway-rules-new-v1"

# Create index if not exists
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # embedding dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Creating index: {index_name}")

# Wait for index to be ready
while not pc.describe_index(index_name).status['ready']:
    print("⏳ Waiting for index to be ready...")
    time.sleep(2)

index = pc.Index(index_name)
print(f"✅ Index '{index_name}' is ready for use.")


✅ Index 'railway-rules-new-v1' is ready for use.


In [9]:
index = pc.Index("railway-rules-new-v1")

### Rule set

In [10]:
rule_set = [
  {
    "id": "rule_01",
    "metadata": {
      "feature": "transaction_complete_time",
      "category": "genuine",
      "weight": "high",
      "rule_text": "If the transaction completion time is greater than 2 minutes, the user behavior strongly aligns with genuine human interaction."
    }
  },
  {
    "id": "rule_02",
    "metadata": {
      "feature": "transaction_complete_time",
      "category": "slightly-fraudulent",
      "weight": "medium",
      "rule_text": "If the transaction completion time is between 1 and 2 minutes, the user shows slightly automated or hurried behavior, possibly semi-fraudulent."
    }
  },
  {
    "id": "rule_03",
    "metadata": {
      "feature": "transaction_complete_time",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the transaction completion time is below 1 minute, the pattern indicates a high chance of automation or bot usage, hence fraudulent."
    }
  },
  {
    "id": "rule_04",
    "metadata": {
      "feature": "avg_transaction_time_28d",
      "category": "genuine",
      "weight": "high",
      "rule_text": "If the user's average transaction time over the last 28 days exceeds 2 minutes, they are more likely a genuine user."
    }
  },
  {
    "id": "rule_05",
    "metadata": {
      "feature": "avg_transaction_time_28d",
      "category": "slightly-fraudulent",
      "weight": "medium",
      "rule_text": "If the average transaction time in the last 28 days is between 1 and 2 minutes, the user shows partially automated patterns, slightly towards fraud."
    }
  },
  {
    "id": "rule_06",
    "metadata": {
      "feature": "avg_transaction_time_28d",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the average transaction time in the last 28 days is below 1 minute, it strongly suggests automation or bot-like efficiency."
    }
  },
  {
    "id": "rule_07",
    "metadata": {
      "feature": "unique_ips_28d",
      "category": "genuine",
      "weight": "high",
      "rule_text": "If the user has used 1 to 3 unique IP addresses in the last 28 days, it reflects a consistent device or network, typically genuine."
    }
  },
  {
    "id": "rule_08",
    "metadata": {
      "feature": "unique_ips_28d",
      "category": "slightly-genuine",
      "weight": "medium",
      "rule_text": "If the user has used 3 to 5 unique IP addresses in the last 28 days, this could indicate mild inconsistency but still within normal user behavior."
    }
  },
  {
    "id": "rule_09",
    "metadata": {
      "feature": "unique_ips_28d",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the user operates from more than 7 IPs in 28 days, it suggests multiple proxies or shared accounts, likely fraudulent."
    }
  },
  {
    "id": "rule_10",
    "metadata": {
      "feature": "time_since_last_txn",
      "category": "genuine",
      "weight": "high",
      "rule_text": "If the time since the last transaction is more than 4 days, the behavior aligns with typical user intervals, indicating genuine activity."
    }
  },
  {
    "id": "rule_11",
    "metadata": {
      "feature": "time_since_last_txn",
      "category": "slightly-genuine",
      "weight": "medium",
      "rule_text": "If the time since the last transaction is between 1 and 4 days, it indicates moderate engagement, possibly genuine but slightly automated."
    }
  },
  {
    "id": "rule_12",
    "metadata": {
      "feature": "time_since_last_txn",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the next transaction occurs within 2 minutes of the previous one, it’s likely automated or fraudulent."
    }
  },
  {
    "id": "rule_13",
    "metadata": {
      "feature": "time_since_last_txn",
      "category": "neutral",
      "weight": "low",
      "rule_text": "If there is no previous transaction (value = -1), no behavioral pattern can be inferred."
    }
  },
  {
    "id": "rule_14",
    "metadata": {
      "feature": "avg_txn_per_week",
      "category": "genuine",
      "weight": "high",
      "rule_text": "If the user makes 0 to 3 transactions per week, this reflects typical user behavior, likely genuine."
    }
  },
  {
    "id": "rule_15",
    "metadata": {
      "feature": "avg_txn_per_week",
      "category": "slightly-fraudulent",
      "weight": "medium",
      "rule_text": "If the user makes 4 to 7 transactions per week, it suggests slightly repetitive but not suspicious behavior."
    }
  },
  {
    "id": "rule_16",
    "metadata": {
      "feature": "avg_txn_per_week",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the user averages more than 7 transactions per week, it indicates unusually frequent activity, possibly fraudulent."
    }
  },
  {
    "id": "rule_17",
    "metadata": {
      "feature": "dependent_tickets_current",
      "category": "genuine",
      "weight": "low",
      "rule_text": "If 1 to 3 dependent tickets are booked in the current transaction, the user is likely genuine."
    }
  },
  {
    "id": "rule_18",
    "metadata": {
      "feature": "dependent_tickets_current",
      "category": "slightly-fraudulent",
      "weight": "low",
      "rule_text": "If there are 4 dependent tickets in the current transaction, the user shows borderline suspicious behavior."
    }
  },
  {
    "id": "rule_19",
    "metadata": {
      "feature": "avg_dependent_ratio_28d",
      "category": "genuine",
      "weight": "medium",
      "rule_text": "If the average dependent ratio is below 0.45, it’s a genuine pattern."
    }
  },
  {
    "id": "rule_20",
    "metadata": {
      "feature": "avg_dependent_ratio_28d",
      "category": "slightly-fraudulent",
      "weight": "medium",
      "rule_text": "If the average dependent ratio is between 0.45 and 0.7, it is slightly leaning towards fraud."
    }
  },
  {
    "id": "rule_21",
    "metadata": {
      "feature": "avg_dependent_ratio_28d",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the average dependent ratio exceeds 0.8, it’s a strong fraud signal as dependency usage is unusually high."
    }
  },
  {
    "id": "rule_22",
    "metadata": {
      "feature": "tickets_current_txn",
      "category": "genuine",
      "weight": "low",
      "rule_text": "If 1 to 3 tickets are booked in the current transaction, it’s typical of genuine personal usage."
    }
  },
  {
    "id": "rule_23",
    "metadata": {
      "feature": "tickets_current_txn",
      "category": "slightly-fraudulent",
      "weight": "low",
      "rule_text": "If 5 tickets are booked in the current transaction, it’s slightly suspicious, possibly for reselling or automation."
    }
  },
  {
    "id": "rule_24",
    "metadata": {
      "feature": "avg_tickets_per_txn",
      "category": "genuine",
      "weight": "high",
      "rule_text": "If the average tickets per transaction is between 1 and 3, the user is genuine."
    }
  },
  {
    "id": "rule_25",
    "metadata": {
      "feature": "avg_tickets_per_txn",
      "category": "slightly-fraudulent",
      "weight": "medium",
      "rule_text": "If the average tickets per transaction is 4, it leans slightly towards fraudulent but still borderline."
    }
  },
  {
    "id": "rule_26",
    "metadata": {
      "feature": "avg_tickets_per_txn",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the average tickets per transaction is 5, it indicates fraudulent or reseller behavior."
    }
  },
  {
    "id": "rule_27",
    "metadata": {
      "feature": "class_ticket_distribution",
      "category": "genuine",
      "weight": "medium",
      "rule_text": "If the user buys tickets spread across multiple classes (1, 2, 3, observation), it reflects normal variation and genuine behavior."
    }
  },
  {
    "id": "rule_28",
    "metadata": {
      "feature": "class_ticket_distribution",
      "category": "genuine",
      "weight": "low",
      "rule_text": "If the user buys only in observation saloon and the total tickets are below 10, it still indicates genuine interest."
    }
  },
  {
    "id": "rule_29",
    "metadata": {
      "feature": "class_ticket_distribution",
      "category": "fraudulent",
      "weight": "high",
      "rule_text": "If the user buys only in observation saloon or class 1 and the number of tickets exceeds 25, it’s a highly fraudulent pattern typical of automated mass booking."
    }
  }
]


### Insert rules into pinecone vector DB

In [13]:
from tqdm import tqdm  # optional, for progress bar

# Function to insert all rules into Pinecone
def insert_rules_into_pinecone(rule_set, index_name, embedding):

    vectors = []

    for rule in tqdm(rule_set, desc="Embedding & preparing rules"):
        rule_id = rule["id"]
        metadata = rule["metadata"]
        rule_text = metadata["rule_text"]

        text_to_embed = f"Feature: {rule['metadata']['feature']}. {rule['metadata']['rule_text']}"
        vector = embedding.embed_query(text_to_embed)   

        # Generate embedding for the rule_text
        # vector = embedding.embed_query(rule_text)

        # Create vector structure for Pinecone
        vectors.append({
            "id": rule_id,
            "values": vector,
            "metadata": metadata
        })

    # Upsert (insert/update) all rules into Pinecone
    index.upsert(vectors=vectors)
    print(f"✅ Successfully inserted {len(rule_set)} rules into Pinecone index '{index_name}'.")

# Run the insertion
insert_rules_into_pinecone(rule_set, index_name, embedding)


Embedding & preparing rules: 100%|██████████| 29/29 [00:00<00:00, 73.41it/s]


✅ Successfully inserted 29 rules into Pinecone index 'railway-rules-new-v1'.


### Create the retriver

In [11]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_existing_index(
    index_name = "railway-rules-new-v1",
    embedding = embedding,
    text_key="rule_text"
)


In [25]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":7})

In [35]:
retrieved_docs = retriever.invoke("dependent tickets booked in current transaction is 2")
retrieved_docs

[Document(id='rule_17', metadata={'category': 'genuine', 'feature': 'dependent_tickets_current', 'weight': 'low'}, page_content='If 1 to 3 dependent tickets are booked in the current transaction, the user is likely genuine.'),
 Document(id='rule_18', metadata={'category': 'slightly-fraudulent', 'feature': 'dependent_tickets_current', 'weight': 'low'}, page_content='If there are 4 dependent tickets in the current transaction, the user shows borderline suspicious behavior.'),
 Document(id='rule_22', metadata={'category': 'genuine', 'feature': 'tickets_current_txn', 'weight': 'low'}, page_content='If 1 to 3 tickets are booked in the current transaction, it’s typical of genuine personal usage.'),
 Document(id='rule_23', metadata={'category': 'slightly-fraudulent', 'feature': 'tickets_current_txn', 'weight': 'low'}, page_content='If 5 tickets are booked in the current transaction, it’s slightly suspicious, possibly for reselling or automation.'),
 Document(id='rule_27', metadata={'category'

In [34]:
# ===========================
# TEST 1: Basic Retriever Test
# ===========================
print("=" * 60)
print("TEST 1: Basic Retriever Test")
print("=" * 60)

test_query = "User completed booking in 2 minutes"
retrieved_docs = retriever.get_relevant_documents(test_query)

print(f"\nQuery: '{test_query}'")
print(f"\nRetrieved {len(retrieved_docs)} documents:\n")

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{'='*50}")
    print(f"Result {i}:")
    print(f"{'='*50}")
    print(f"Content: {doc.page_content}")
    print(f"\nMetadata:")
    for key, value in doc.metadata.items():
        print(f"  - {key}: {value}")



TEST 1: Basic Retriever Test

Query: 'User completed booking in 2 minutes'

Retrieved 3 documents:


Result 1:
Content: If the transaction completion time is between 1 and 2 minutes, the user shows slightly automated or hurried behavior, possibly semi-fraudulent.

Metadata:
  - category: slightly-fraudulent
  - feature: transaction_complete_time
  - weight: medium

Result 2:
Content: If the transaction completion time is greater than 2 minutes, the user behavior strongly aligns with genuine human interaction.

Metadata:
  - category: genuine
  - feature: transaction_complete_time
  - weight: high

Result 3:
Content: If the user's average transaction time over the last 28 days exceeds 2 minutes, they are more likely a genuine user.

Metadata:
  - category: genuine
  - feature: avg_transaction_time_28d
  - weight: high


### Connect with the LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [36]:
# transaction data
transaction_data = {
    "transaction_complete_time": 0.9,                   # minutes
    "average_transaction_time_28d": 1.5,                # minutes
    "unique_ips_28d": 8,
    "time_since_last_transaction": 1,                # days (~43 mins)
    "average_transactions_per_week": 7,
    "dependent_tickets_current": 4,
    "average_dependent_tickets_ratio_28d": 0.7,
    "tickets_current": 5,
    "average_tickets_per_transaction": 4,
    "tickets_class_1": 11,
    "tickets_class_2": 0,
    "tickets_class_3": 0,
    "tickets_observation_salon": 15
}


In [38]:
# convert transaction data to text
def transaction_to_text(data: dict) -> str:
    text = f"""
    The user completed the current booking in {data['transaction_complete_time']} minutes.
    In the last 28 days, the user's average transaction time was {data['average_transaction_time_28d']} minutes.
    The user used {data['unique_ips_28d']} unique IP addresses.
    Time since last transaction: {data['time_since_last_transaction']} days.
    Average transactions per week: {data['average_transactions_per_week']}.
    Dependent tickets in current transaction: {data['dependent_tickets_current']}.
    Average dependent tickets ratio in last 28 days: {data['average_dependent_tickets_ratio_28d']}.
    Tickets in current transaction: {data['tickets_current']}.
    Average tickets per transaction: {data['average_tickets_per_transaction']}.
    Tickets per class:
        Class 1 = {data['tickets_class_1']},
        Class 2 = {data['tickets_class_2']},
        Class 3 = {data['tickets_class_3']},
        Observation saloon = {data['tickets_observation_salon']}.
    """
    return text.strip()


In [39]:
# retrieving rules from pinecone based on transaction data
query_text = transaction_to_text(transaction_data)
retrieved_rules = retriever.get_relevant_documents(query_text)

In [19]:
# system prompt
system_prompt = """
You are an intelligent fraud detection assistant for a railway ticketing system.
You are given:
1. A user's current transaction details.
2. The most relevant behavioral rules from a rule database (some indicate fraudulent, some genuine behavior).

Your task:
- Analyze the transaction details in light of the retrieved rules.
- Explain which rules apply most strongly.
- Conclude whether the user is more likely a "Genuine User" or a "Fraudulent User".
- Give a short justification (2–3 sentences) referencing the rules.

Be objective and concise in your final decision.
"""


In [40]:
# LLM result
from langchain.schema import SystemMessage, HumanMessage

# Prepare retrieved rule text
rules_text = "\n\n".join([f"Rule {i+1}: {doc.page_content}" for i, doc in enumerate(retrieved_rules)])

# Create the conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=f"Transaction details:\n{query_text}\n\nRelevant rules:\n{rules_text}")
]

# Run the model
response = llm.invoke(messages)

print("\n================= RESULT =================")
print(response.content)



================= RESULT =================
**Analysis:**

The user's current transaction has 5 tickets, which aligns with Rule 1, suggesting potentially fraudulent or reseller behavior. The average transaction time of 0.9 minutes is faster than their 28-day average of 1.5 minutes, which, combined with Rule 4, indicates partially automated patterns leaning towards fraud. The user has purchased 4 dependent tickets, exceeding the average dependent ticket ratio of 0.7, which could be suspicious. However, the total number of tickets in this transaction (5) is not excessively high, and the user has not exclusively booked in the observation saloon, so Rule 5 and Rule 7 are not strongly applicable. Rule 6 is not applicable as the current transaction has 5 tickets.

**Decision:**

Fraudulent User

**Justification:**

The current transaction of 5 tickets directly matches Rule 1, indicating potential reseller or fraudulent activity. Additionally, the faster-than-average transaction time, as per 